In [34]:
import os
import pandas as pd
import numpy as np
import mysql.connector
from dotenv import load_dotenv
import seaborn as sns
from scipy import stats

In [24]:
import matplotlib.pyplot as plt
plt.rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False

In [9]:
# DB 연결
load_dotenv()

conn = mysql.connector.connect(
    host=os.getenv("DB_HOST"),
    user=os.getenv("DB_USER"),
    password=os.getenv("DB_PASSWORD"),
    database=os.getenv("DB_DATABASE")
)

cursor = conn.cursor()

In [12]:
# 쿼리 정의 (업종별 월평균 소비 EDA용)
query = """
SELECT
  r.sigungu_code AS city_code,
  ic.sub_category AS industry,
  DATE_FORMAT(ac.ymd, '%Y-%m') AS ym,
  SUM(ac.amt) AS total_amt,
  SUM(ac.cnt) AS total_cnt
FROM aggregated_consumption ac
JOIN regions r ON ac.region_id = r.id
JOIN industry_categories ic ON ac.industry_id = ic.id
GROUP BY r.sigungu_code, ic.sub_category, ym
ORDER BY r.sigungu_code, ym, total_amt DESC;
"""

df = pd.read_sql(query, conn)
conn.close()

df

/var/folders/36/pdpmp_4s2wv687klck6zglrm0000gn/T/ipykernel_75323/2379537012.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,city_code,industry,ym,total_amt,total_cnt
0,41111,종합소매점,2025-01,3.283674e+10,1760589.0
1,41111,연료판매,2025-01,1.442784e+10,306634.0
2,41111,차량관리/서비스,2025-01,1.341227e+10,278535.0
3,41111,기업,2025-01,1.058326e+10,242231.0
4,41111,한식,2025-01,8.500557e+09,278578.0
...,...,...,...,...,...
3709,41650,부동산,2025-03,1.838844e+06,123.0
3710,41650,금융상품/서비스,2025-03,1.320019e+06,109.0
3711,41650,유아용품,2025-03,1.154458e+06,20.0
3712,41650,교통서비스,2025-03,6.923070e+05,4.0


In [13]:
df.head()

,city_code,industry,ym,total_amt,total_cnt
0,41111,종합소매점,2025-01,3.283674e+10,1760589.0
1,41111,연료판매,2025-01,1.442784e+10,306634.0
2,41111,차량관리/서비스,2025-01,1.341227e+10,278535.0
3,41111,기업,2025-01,1.058326e+10,242231.0
4,41111,한식,2025-01,8.500557e+09,278578.0


In [37]:
df.count()

city_code    3714
industry     3714
ym           3714
total_amt    3714
total_cnt    3714
dtype: int64

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3714 entries, 0 to 3713
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   city_code  3714 non-null   object 
 1   industry   3714 non-null   object 
 2   ym         3714 non-null   object 
 3   total_amt  3714 non-null   float64
 4   total_cnt  3714 non-null   float64
dtypes: float64(2), object(3)
memory usage: 145.2+ KB


In [15]:
df.describe()

,total_amt,total_cnt
count,3.714000e+03,3.714000e+03
mean,3.391712e+09,9.946350e+04
std,9.402386e+09,3.702021e+05
min,3.590000e+03,2.000000e+00
25%,1.062541e+08,1.528250e+03
50%,7.910113e+08,1.414550e+04
75%,2.640493e+09,6.434850e+04
max,1.608099e+11,8.632500e+06


In [36]:
# 도시 간 소비액 차이
cities_data = [group['total_amt'].values for name, group in df.groupby('city_code')]
f_stat_city, p_value_city = stats.f_oneway(*cities_data)

print("=== 1. 도시 간 소비액 차이 검정 ===")
print(f"F-통계량: {f_stat_city:.4f}")
print(f"p-value: {p_value_city:.4f}")
print(f"결과: {'유의한 차이 있음' if p_value_city < 0.05 else '유의한 차이 없음'}")

# 업종 간 소비액 차이
industries_data = [group['total_amt'].values for name, group in df.groupby('industry')]
f_stat_industry, p_value_industry = stats.f_oneway(*industries_data)

print("\n=== 2. 업종 간 소비액 차이 검정 ===")
print(f"F-통계량: {f_stat_industry:.4f}")
print(f"p-value: {p_value_industry:.4f}")
print(f"결과: {'유의한 차이 있음' if p_value_industry < 0.05 else '유의한 차이 없음'}")

# 특정 도시-업종 조합의 유의성 검정
print("\n=== 3. 특정 도시-업종 조합의 유의성 검정 ===")

# 상위 조합 vs 전체 평균 비교
top_combination = df.groupby(['city_code', 'industry'])['total_amt'].mean().idxmax()
top_city, top_industry = top_combination

# 해당 조합의 데이터
target_data = df[(df['city_code'] == top_city) & (df['industry'] == top_industry)]['total_amt']

# 다른 모든 데이터
other_data = df[~((df['city_code'] == top_city) & (df['industry'] == top_industry))]['total_amt']

# t-검정
t_stat, p_value_t = stats.ttest_ind(target_data, other_data)

print(f"최고 조합: 도시 {top_city}, 업종 {top_industry}")
print(f"해당 조합 평균: {target_data.mean():.2f}")
print(f"다른 조합 평균: {other_data.mean():.2f}")
print(f"t-통계량: {t_stat:.4f}")
print(f"p-value: {p_value_t:.4f}")
print(f"결과: {'유의하게 높음' if p_value_t < 0.05 and t_stat > 0 else '유의한 차이 없음'}")

=== 1. 도시 간 소비액 차이 검정 ===
F-통계량: 8.3896
p-value: 0.0000
결과: 유의한 차이 있음

=== 2. 업종 간 소비액 차이 검정 ===
F-통계량: 65.4386
p-value: 0.0000
결과: 유의한 차이 있음

=== 3. 특정 도시-업종 조합의 유의성 검정 ===
최고 조합: 도시 41590, 업종 종합소매점
해당 조합 평균: 148476708792.67
다른 조합 평균: 3274423863.85
t-통계량: 29.7510
p-value: 0.0000
결과: 유의하게 높음
